In [0]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score, KFold
from sklearn import preprocessing as prep
from sklearn import utils

In [0]:
dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
data = pd.read_csv(dataset_url,  header=None)

In [90]:
feature_names_raw = """1) Alcohol
2) Malic acid
3) Ash
4) Alcalinity of ash
5) Magnesium
6) Total phenols
7) Flavanoids
8) Nonflavanoid phenols
9) Proanthocyanins
10)Color intensity
11)Hue
12)OD280/OD315 of diluted wines
13)Proline"""
feature_names = ['Class']
for afeat in feature_names_raw.split('\n'):
    feature_names.append(afeat.replace(") ",")").split(")")[1])

feature_names

['Class',
 'Alcohol',
 'Malic acid',
 'Ash',
 'Alcalinity of ash',
 'Magnesium',
 'Total phenols',
 'Flavanoids',
 'Nonflavanoid phenols',
 'Proanthocyanins',
 'Color intensity',
 'Hue',
 'OD280/OD315 of diluted wines',
 'Proline']

In [91]:
data.columns = feature_names
data

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [0]:
def get_outliers_count(data, outliers_level = 2):
    feature_names = data.columns
    params = {afeat:{'median':data[afeat].median(),'count':0} for afeat in feature_names}
    for afeat in feature_names:
        params[afeat]['count'] = data[afeat].apply(lambda x: 0 if x < params[afeat]['median']*outliers_level else 1).sum()
        print(afeat, params[afeat]['count'])


In [93]:
get_outliers_count(data, 2)

Class 0
Alcohol 0
Malic acid 28
Ash 0
Alcalinity of ash 0
Magnesium 0
Total phenols 0
Flavanoids 1
Nonflavanoid phenols 0
Proanthocyanins 2
Color intensity 11
Hue 0
OD280/OD315 of diluted wines 0
Proline 7


In [94]:
X = data.drop(['Class'], axis=1)
y= data.Class
X.shape, y.shape

((178, 13), (178,))

In [95]:
from sklearn.neighbors import KNeighborsClassifier
clf  = KNeighborsClassifier(n_neighbors=3)
clf.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [0]:
# from sklearn.model_selection import cross_val_score, KFold

clf_kfold = KFold(n_splits = 5, shuffle = True, random_state = 42)


scores = cross_val_score(clf, X, y, cv=KFold(n_splits = 5, shuffle = True, random_state = 42))

In [0]:
def get_cross_val_scores_knn(n_neighbors):
    scores = {}
    for neighbors in range(1,n_neighbors+1):
        clf  = KNeighborsClassifier(n_neighbors=neighbors)
        clf.fit(X, y)
        scores[neighbors] = cross_val_score(clf, X, y, cv=KFold(n_splits = 5, shuffle = True, random_state = 42))
    return scores

In [0]:
scores = get_cross_val_scores_knn(50)
df_scores = pd.DataFrame(scores).T.mean(axis=1)

In [99]:
df_scores.idxmax(), df_scores[df_scores.idxmax()]

(1, 0.7304761904761905)

In [100]:
X

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [0]:
X_old = X.copy()
from sklearn import preprocessing
X = pd.DataFrame(preprocessing.scale(X), columns=X_old.columns)


In [0]:
scores = get_cross_val_scores_knn(50)
df_scores = pd.DataFrame(scores).T.mean(axis=1)

In [103]:
df_scores.idxmax(), df_scores[df_scores.idxmax()]

(29, 0.9776190476190475)

In [0]:
# from sklearn import datasets
data = datasets.load_boston()

In [112]:
print(data.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [116]:
data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [0]:
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame(data.target, columns = ['Target'])

In [151]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,6.32e-03,18.0,2.31,0.0,0.54,6.58,65.2,4.09,1.0,296.0,15.3,396.90,4.98
1,2.73e-02,0.0,7.07,0.0,0.47,6.42,78.9,4.97,2.0,242.0,17.8,396.90,9.14
2,2.73e-02,0.0,7.07,0.0,0.47,7.18,61.1,4.97,2.0,242.0,17.8,392.83,4.03
3,3.24e-02,0.0,2.18,0.0,0.46,7.00,45.8,6.06,3.0,222.0,18.7,394.63,2.94
4,6.91e-02,0.0,2.18,0.0,0.46,7.15,54.2,6.06,3.0,222.0,18.7,396.90,5.33


In [0]:
X = pd.DataFrame(prep.scale(X), columns = data.feature_names)
label_encoder = prep.LabelEncoder()
y = label_encoder.fit_transform(y.Target)

In [181]:
print(utils.multiclass.type_of_target(y))

multiclass


In [147]:
X.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02,5.06e+02
mean,6.91e-18,7.66e-17,-2.24e-16,-1.18e-15,-2.90e-16,-2.41e-18,8.35e-17,-7.46e-18,-6.54e-17,-8.03e-17,1.58e-15,-5.05e-17,-5.18e-17
std,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00
min,-4.20e-01,-4.88e-01,-1.56e+00,-2.73e-01,-1.47e+00,-3.88e+00,-2.34e+00,-1.27e+00,-9.83e-01,-1.31e+00,-2.71e+00,-3.91e+00,-1.53e+00
25%,-4.11e-01,-4.88e-01,-8.68e-01,-2.73e-01,-9.13e-01,-5.69e-01,-8.37e-01,-8.06e-01,-6.38e-01,-7.68e-01,-4.88e-01,2.05e-01,-7.99e-01
50%,-3.91e-01,-4.88e-01,-2.11e-01,-2.73e-01,-1.44e-01,-1.08e-01,3.17e-01,-2.79e-01,-5.23e-01,-4.65e-01,2.75e-01,3.81e-01,-1.81e-01
75%,7.40e-03,4.88e-02,1.02e+00,-2.73e-01,5.99e-01,4.83e-01,9.07e-01,6.62e-01,1.66e+00,1.53e+00,8.07e-01,4.34e-01,6.03e-01
max,9.93e+00,3.80e+00,2.42e+00,3.67e+00,2.73e+00,3.56e+00,1.12e+00,3.96e+00,1.66e+00,1.80e+00,1.64e+00,4.41e-01,3.55e+00


In [182]:
y

array([137, 113, 196, 193, 203, 165, 126, 156,  64,  87,  52,  87, 114,
       102,  80,  97, 128,  73, 100,  80,  40,  94,  54,  48,  57,  42,
        65,  50,  82, 108,  32,  48,  36,  35,  39,  87,  98, 108, 144,
       176, 197, 153, 149, 144, 110,  91,  98,  65,  47,  92,  95, 103,
       146, 131,  87, 200, 144, 181, 130,  94,  85,  59, 119, 146, 189,
       132,  92, 117,  72, 107, 139, 114, 125, 131, 138, 111,  98, 106,
       110, 101, 159, 136, 145, 126, 136, 153, 122, 119, 133, 165, 123,
       117, 126, 146, 104, 162, 111, 211, 219, 191, 157, 152,  84,  91,
        99,  93,  93, 102,  96,  92, 114, 125,  86,  85,  83,  81, 110,
        90, 102,  91, 117, 101, 103,  71,  86, 111,  58,  61,  78,  46,
        90,  94, 127,  82,  57,  79,  72,  69,  37,  76,  43,  47,  38,
        57,  25,  41,  57,  49,  76,  56, 112,  94,  55,  92,  68,  57,
        35, 213, 140, 130, 155, 228, 228, 228, 124, 146, 228, 135, 135,
       120,  72,  89, 128, 133, 123, 168, 129, 143, 171, 207, 21

In [192]:
def get_cross_val_scores_knn_parametric(parameter_name, parameter_values, **kwargs):
    scores = {}
    params = {}
    params.update(kwargs)
    print("testing parameters:", kwargs)
    for test_value in parameter_values:
        params[parameter_name] = test_value
        clf  = KNeighborsClassifier(**params)
        clf.fit(X, y)
        scores[test_value] = cross_val_score(clf, X, y, cv=KFold(n_splits = 5, shuffle = True, random_state = 42), scoring='neg_mean_squared_error')
    return scores

parameter_name = 'p'
parameter_values = np.linspace(1, 10, 200)
model_parameters = {'n_neighbors': 5, 'weights': 'distance', 'metric': 'minkowski'}
result = get_cross_val_scores_knn_parametric(parameter_name, parameter_values, **model_parameters)

testing parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'minkowski'}


In [0]:
df_scores = pd.DataFrame(result).T

In [0]:
df_scores = df_scores.mean(axis=1)

In [210]:
df_scores.idxmax(), df_scores[df_scores.idxmax()]

(1.0452261306532664, -746.4670743544942)